###### Importing libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from IPython.display import display
import shutil
import re

###### Extracting all actor links from Wikipedia

In [2]:
urls = ["https://en.wikipedia.org/wiki/List_of_Indian_film_actors", "https://en.wikipedia.org/wiki/List_of_Indian_film_actresses"] 

base = "https://en.wikipedia.org"
actor_urls = []

for url in urls:    
    source = requests.get(url).text
    soup = BeautifulSoup(source, 'lxml')
    
    divs = soup.find_all('div', class_="div-col columns column-width")
    for div in divs:
        lis = div.findChildren("a", recursive="False")
        for li in lis:
            actor_urls.append(base+li['href'])
            
print("Total {} actors got.".format(len(actor_urls)))
display(actor_urls[:5])

Total 1732 actors got.


['https://en.wikipedia.org/wiki/A._K._Hangal',
 'https://en.wikipedia.org/wiki/Aadhi_(actor)',
 'https://en.wikipedia.org/wiki/Aadi_(Telugu_actor)',
 'https://en.wikipedia.org/wiki/Aamir_Khan',
 'https://en.wikipedia.org/wiki/Aashish_Chaudhary']

###### Download images from Google

In [3]:
search_url = "https://www.google.com/search?hl=en&site=imghp&tbm=isch&tbs=isz:l&q="

def google_image(name, fn):
    headers= {"User.Agent":"Chrome/80.0.3987.122"}
    url = search_url+name
    
    req= requests.get(url,headers=headers)
    html=req.content
    
    a = requests.get(url).text
    result_url=""
    b=a.find("http://t1.gstatic.com")
    for i in range(b,b+1000):
        if a[i]=='"':
            break
        result_url+=a[i]
    #print(result_url)
    response = requests.get(result_url, stream=True)
    
    file = open("./Images/{}.jpg".format(fn), 'wb')
    
    response.raw.decode_content = True
    shutil.copyfileobj(response.raw, file)
    del response

###### Download images from Wikipedia only

In [4]:
def download_image(url, fn):
    response = requests.get(url, stream=True)
    
    file = open("./Images/{}.jpg".format(fn), 'wb')
    
    response.raw.decode_content = True
    shutil.copyfileobj(response.raw, file)
    del response

###### Get actor age

In [44]:
def get_age(query):
    query += " age"
    query = query.replace(' ', '+')
    URL = f"https://google.com/search?q={query}"
    
    headers = {"User.Agent": "Mozilla/5.0 (Linux; Android 7.0; SM-G930V Build/NRD90M) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.125 Mobile Safari/537.36"}
    response = requests.get(URL, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "lxml")
    else:
        # print("{} - not found".format(URL))
        return "Unavailable"
    #print(soup.prettify())
    
    div = soup.find("div", class_="BNeawe").find("div", class_="BNeawe")
    if div:
        if len(div.text) < 50:
            return div.text
        else:
            return "Unavailable"
    else:
        return "Unavailable"

###### Get actor heights

In [36]:
def get_height(query):
    query += " height"
    query = query.replace(' ', '+')
    URL = f"https://google.com/search?q={query}"
    
    headers = {"User.Agent": "Mozilla/5.0 (Linux; Android 7.0; SM-G930V Build/NRD90M) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.125 Mobile Safari/537.36"}
    response = requests.get(URL, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "lxml")
    else:
        # print("{} - not found".format(URL))
        return "Unavailable"
    #print(soup.prettify())
    
    div = soup.find('div', class_="BNeawe iBp4i AP7Wnd")
    if div:
        txt = div.text
        if len(txt) < 10:
            return txt
        else:
            return txt.split('.')[0]
    else:
        table = soup.find("table", class_="LnMnt")
        if table:
            trs = table.find_all('tr')
            for tr in trs:
                if tr.find_all('td')[0].text == 'HEIGHT':
                    return tr.find_all('td')[1].text.split('-')[2][1:7]
                    break
        else:
            div = soup.find('div', class_="BNeawe").find('div', class_="BNeawe")
            if div:
                txt = div.text
                if len(txt) > 10:
                    return txt.split(".")[0]
                else:
                    return div.text
            else:
                return "Unavailable"

###### Check if actor is dead or alive

In [21]:
def is_alive(query):
    query = "is "+ query + " alive"
    query = query.replace(' ', '+')
    URL = f"https://google.com/search?q={query}"
    
    headers = {"User-Agent": "Mozilla/5.0 (Linux; Android 7.0; SM-G930V Build/NRD90M) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.125 Mobile Safari/537.36"}
    response = requests.get(URL, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "lxml")
    else:
        # print("{} - not found".format(URL))
        return "Unavailable"
    soup = BeautifulSoup(response.text, "lxml")
    print(soup.prettify())
    d = soup.find(text="Deceased")
    c = soup.find(text="Cause of death")
    print(d, c)
    
    if d or c:
        return "No"
    else:
        return "Yes"

###### Get all information about each actor

In [37]:
data = {'name':[], 'image':[], 'age':[], 'height':[], 'description':[]}

for i, actor_url in enumerate(actor_urls, 1):
    #print(actor_url)
    source = requests.get(actor_url).text
    soup = BeautifulSoup(source, 'lxml')
    
    name = soup.find('h1', id="firstHeading").text
    
    try:
        paras = soup.find('div', class_="mw-parser-output").find_all('p')
        for para in paras:
            txt = para.text
            if len(txt) > 3: 
                txt = re.sub(r'\[.*?\]', '', txt)
                txt = txt.strip()
                des['description'].append(txt)
                #print("\r{}".format(name), end="\r")
                break
    except AttributeError:
        des['description'].append("NaN")
    
    box = soup.find('table', class_="infobox")
    if box:
        img = box.find("a", class_="image")
        if img:
            img = "https:"+img.find_all("img")[0]['src']
            download_image(img, i)
        else:
            google_image(name+' '+box.find('tr').text, i)
    else:
        google_image(name, i)
        
    age = get_age(name)
    
    height = get_height(name)
    
    data["name"].append(name)
    data["image"].append(str(i)+'.jpg')
    data["age"].append(age)
    data["height"].append(height)
    
    print("{}/{} Actors downloaded. [{}, {}, {}]".format(i, len(actor_urls), name, age, height))

df = pd.DataFrame(data)
df.to_csv('./data.csv', encoding='utf-8', index=False)

1/1732 Actors downloaded. [A. K. Hangal, 98 years, 1.63 m]
2/1732 Actors downloaded. [Aadhi Pinisetty, 37 years, 1.72 m]
3/1732 Actors downloaded. [Aadi (actor), 30 years, 1.68 m]
4/1732 Actors downloaded. [Aamir Khan, 55 years, 1.63 m]
5/1732 Actors downloaded. [Ashish Chaudhary, 41 years, 1.78 m]
6/1732 Actors downloaded. [Abbas (actor), 44 years, Abbas Height is 175 Cm and 5' 8” inches]
7/1732 Actors downloaded. [Abhay Deol, 44 years, 1.8 m]
8/1732 Actors downloaded. [Abhijith, 56 years, 1.75 m]
9/1732 Actors downloaded. [Abhishek Bachchan, 44 years, 1.86 m]
10/1732 Actors downloaded. [Abhishek Chatterjee, 55 years, 1.75 m]
11/1732 Actors downloaded. [Ambareesh, 66 years, 1.73 m]
12/1732 Actors downloaded. [Abhimanyu Singh, 44 years, None]
13/1732 Actors downloaded. [Abijeet (actor), 31 years, 1.78 m]
14/1732 Actors downloaded. [Abir Chatterjee, 39 years, 1.88 m]
15/1732 Actors downloaded. [Achyuth Kumar, 54 years, Achyuth Kumar Height is 173 Cm and 5' 8” Inches]
16/1732 Actors down

131/1732 Actors downloaded. [Chetan Chandra, 32 years, 1.72 m]
132/1732 Actors downloaded. [Chetan Kumar, 36 years, 1.72 m]
133/1732 Actors downloaded. [Chhabi Biswas, 61 years, Unavailable]
134/1732 Actors downloaded. [Chi. Guru Dutt, Unavailable, None]
135/1732 Actors downloaded. [Chinni Jayanth, 59 years, 1.73 m]
136/1732 Actors downloaded. [Chikkanna, 35 years, 1.71 m]
137/1732 Actors downloaded. [Chiranjeevi, 64 years, 1.71 m]
138/1732 Actors downloaded. [Chiranjeevi Sarja, 39 years, 1.76 m]
139/1732 Actors downloaded. [V. Nagaiah, 69 years, Unavailable]
140/1732 Actors downloaded. [Chunky Pandey, 57 years, 1.8 m]
141/1732 Actors downloaded. [Crazy Mohan, 66 years, 1.74 m]
142/1732 Actors downloaded. [Cyrus Broacha, 48 years, None]
143/1732 Actors downloaded. [Cyrus Sahukar, 39 years, None]
144/1732 Actors downloaded. [Dada Kondke, 65 years, Unavailable]
145/1732 Actors downloaded. [Dalip Tahil, 67 years, 1.73 m]
146/1732 Actors downloaded. [Danny Denzongpa, 72 years, None]
147/17

260/1732 Actors downloaded. [Jatin Bora, 50 years, None]
261/1732 Actors downloaded. [Jaya Prakash Reddy, 74 years, 1.85 m]
262/1732 Actors downloaded. [Jayadev Mohan, Unavailable, Unavailable]
263/1732 Actors downloaded. [Jayam Ravi, 39 years, 1.75 m]
264/1732 Actors downloaded. [Jayan, 41 years, 1.83 m]
265/1732 Actors downloaded. [Jayant (actor), 59 years, 1.81 m]
266/1732 Actors downloaded. [Jayaram, 55 years, 1.83 m]
267/1732 Actors downloaded. [Karthik Jayaram, 40 years, 1.85 m]
268/1732 Actors downloaded. [Jayasurya, 41 years, 1.73 m]
269/1732 Actors downloaded. [Jeet (actor), 41 years, 1.78 m]
270/1732 Actors downloaded. [Jeetendra, 78 years, 1.75 m]
271/1732 Actors downloaded. [Jeeva (Telugu actor), 67 years, None]
272/1732 Actors downloaded. [Jeevan (actor), 71 years, None]
273/1732 Actors downloaded. [Jhanak Shukla, 24 years, 1.57 m]
274/1732 Actors downloaded. [Jiiva, 36 years, 1.82 m]
275/1732 Actors downloaded. [Jimmy Sheirgill, 49 years, 1.75 m]
276/1732 Actors downloade

388/1732 Actors downloaded. [Mohan Babu, 70 years, 1.85 m]
389/1732 Actors downloaded. [Mohanlal, 59 years, 1.72 m]
390/1732 Actors downloaded. [Mohan Joshi, 74 years, None]
391/1732 Actors downloaded. [Mohan (actor), 63 years, 1.8 m ]
392/1732 Actors downloaded. [Mohan Shankar, 46 years, 1.68 m]
393/1732 Actors downloaded. [Mohit Madaan, 32 years, Unavailable]
394/1732 Actors downloaded. [Mohit Marwah, 35 years, 1.7 m]
395/1732 Actors downloaded. [Mohit Raina, 37 years, 1.8 m]
396/1732 Actors downloaded. [Mohnish Bahl, 58 years, None]
397/1732 Actors downloaded. [Motilal (actor), 54 years, Unavailable]
398/1732 Actors downloaded. [Mukesh (actor), 64 years, 1.78 m]
399/1732 Actors downloaded. [Mukesh Khanna, 61 years, 1.88 m]
400/1732 Actors downloaded. [Mukkamala, 67 years, Unavailable]
401/1732 Actors downloaded. [Mukri, 78 years, 1.5 m ]
402/1732 Actors downloaded. [Mukhyamantri Chandru, 66 years, Unavailable]
403/1732 Actors downloaded. [Mukul Dev, 50 years, 1.72 m]
404/1732 Actors

514/1732 Actors downloaded. [Rajinikanth, 69 years, 1.75 m]
515/1732 Actors downloaded. [Ram Kapoor, 46 years, 1.75 m]
516/1732 Actors downloaded. [Rajeev Verma, 70 years, Unavailable]
517/1732 Actors downloaded. [Rajiv Kanakala, Unavailable, 175 Cm]
518/1732 Actors downloaded. [Rajiv Kapoor, 57 years, 1.7 m]
519/1732 Actors downloaded. [Rajkumar (actor), 76 years, 1.7 m]
520/1732 Actors downloaded. [Rajneesh Duggal, 40 years, None]
521/1732 Actors downloaded. [Rajpal Yadav, 49 years, 1.6 m]
522/1732 Actors downloaded. [Raju Srivastav, 56 years, None]
523/1732 Actors downloaded. [Rakesh Bedi, 65 years, None]
524/1732 Actors downloaded. [Rakesh Roshan, 70 years, 1.8 m]
525/1732 Actors downloaded. [Ramakrishna (Kannada actor), About 66 years, Unavailable]
526/1732 Actors downloaded. [Ramki, 58 years, None]
527/1732 Actors downloaded. [Ram Charan, 35 years, 1.74 m]
528/1732 Actors downloaded. [Ramesh Aravind, 55 years, 1.67 m]
529/1732 Actors downloaded. [Ghattamaneni Ramesh Babu, 54 year

...

1153/1732 Actors downloaded. [Madhura Naik, 32 years, None]
1154/1732 Actors downloaded. [Madonna Sebastian, 27 years, 1.68 m]
1155/1732 Actors downloaded. [Mahasweta Ray, 57 years, 1.68 m]
1156/1732 Actors downloaded. [Mahalakshmi (Kannada actress), Unavailable, None]
1157/1732 Actors downloaded. [Mahie Gill, 44 years, 1.57 m]
1158/1732 Actors downloaded. [Mahika Sharma, 25 years, None]
1159/1732 Actors downloaded. [Mahima Chaudhry, 46 years, 1.65 m]
1160/1732 Actors downloaded. [Mahira Khan, 35 years, 1.65 m]
1161/1732 Actors downloaded. [Mahua Roychoudhury, 26 years, Unavailable]
1162/1732 Actors downloaded. [Mala Sinha, 83 years, 1.57 m]
1163/1732 Actors downloaded. [Malashri, 46 years, 173 cm tall]
1164/1732 Actors downloaded. [Malavika (actress), 40 years, 1.65 m]
1165/1732 Actors downloaded. [Malaika Arora, 46 years, 1.6 m]
1166/1732 Actors downloaded. [Malavika Avinash, 44 years, 5 feet 3 inches]
1167/1732 Actors downloaded. [Malavika Nair (Indian actress), About 24 years, None

1281/1732 Actors downloaded. [Nikki Galrani, 27 years, 1.59 m]
1282/1732 Actors downloaded. [Meera Chopra, 36 years, 1.57 m]
1283/1732 Actors downloaded. [Nimisha Sajayan, 23 years, 5 ft 4 in]
1284/1732 Actors downloaded. [Nimmi, 87 years, 1.57 m]
1285/1732 Actors downloaded. [Nirmala Chennappa, Unavailable, Unavailable]
1286/1732 Actors downloaded. [Nirupa Roy, 73 years, 1.6 m]
1287/1732 Actors downloaded. [Nirmalamma, 82 years, 1.86 m]
1288/1732 Actors downloaded. [Nirosha, 49 years, 5 feet 6 inches]
1289/1732 Actors downloaded. [Nisha Agarwal, 30 years, 1.61 m]
1290/1732 Actors downloaded. [Priyanka Kothari, 36 years, 1.57 m]
1291/1732 Actors downloaded. [Nisha Ravikrishnan, Unavailable, None]
1292/1732 Actors downloaded. [Nishi (actress), Unavailable, None]
1293/1732 Actors downloaded. [Nithya Das, 38 years, 5 feet 8 inches]
1294/1732 Actors downloaded. [Nithya Menen, 32 years, 1.54 m]
1295/1732 Actors downloaded. [Nivedita Jain, 19 years, None]
1296/1732 Actors downloaded. [Nivedi

1408/1732 Actors downloaded. [Rashmi (actress), 32 years, 1.6 m]
1409/1732 Actors downloaded. [Rashami Desai, 34 years, 1.6 m]
1410/1732 Actors downloaded. [Rashmi Gautam, 32 years, 1.65 m]
1411/1732 Actors downloaded. [Rashmika Mandanna, 24 years, 1.61 m]
1412/1732 Actors downloaded. [Ratan Rajput, 33 years, None]
1413/1732 Actors downloaded. [Rati Agnihotri, 59 years, 1.64 m]
1414/1732 Actors downloaded. [Rati Pandey, 37 years, 1.65 m]
1415/1732 Actors downloaded. [Ratna Pathak, 63 years, None]
1416/1732 Actors downloaded. [Raveena Tandon, 45 years, 1.63 m]
1417/1732 Actors downloaded. [Reena Roy, 63 years, 1.63 m]
1418/1732 Actors downloaded. [Reema Lagoo, 58 years, None]
1419/1732 Actors downloaded. [Reemma Sen, 38 years, 1.65 m]
1420/1732 Actors downloaded. [Regina Cassandra, 29 years, 1.6 m]
1421/1732 Actors downloaded. [Rekha, 65 years, 1.65 m]
1422/1732 Actors downloaded. [Rekha (South Indian actress), 49 years, 5 feet 6 inches]
1423/1732 Actors downloaded. [Rekha Das, Unavaila

1537/1732 Actors downloaded. [Shilpi Sharma, 37 years, 1.7 m]
1538/1732 Actors downloaded. [Shruti Naidu, Unavailable, Unavailable]
1539/1732 Actors downloaded. [Shylaja Nag, Unavailable, Unavailable]
1540/1732 Actors downloaded. [Shweta Basu Prasad, 29 years, 1.65 m]
1541/1732 Actors downloaded. [Shweta Bhardwaj, 34 years, None]
1542/1732 Actors downloaded. [Shweta Menon, 46 years, 1.7 m]
1543/1732 Actors downloaded. [Shweta Gulati, 40 years, 1.6 m]
1544/1732 Actors downloaded. [Shweta Tiwari, 39 years, 1.73 m]
1545/1732 Actors downloaded. [Shobana, 50 years, 1.68 m]
1546/1732 Actors downloaded. [Shobhna Samarth, 85 years, None]
1547/1732 Actors downloaded. [Shraddha Das, 33 years, 1.63 m]
1548/1732 Actors downloaded. [Shraddha Kapoor, 33 years, 1.62 m]
1549/1732 Actors downloaded. [Shraddha Srinath, 29 years, 5 ft 4 in]
1550/1732 Actors downloaded. [Shravanthi Sainath, Unavailable, Unavailable]
1551/1732 Actors downloaded. [Shruti Haasan, 34 years, 1.65 m]
1552/1732 Actors downloaded

1661/1732 Actors downloaded. [Tia Bajpai, 35 years, None]
1662/1732 Actors downloaded. [Tina Ambani, 63 years, 1.65 m]
1663/1732 Actors downloaded. [Tina Datta, 33 years, 1.55 m]
1664/1732 Actors downloaded. [Tisca Chopra, 46 years, 1.63 m]
1665/1732 Actors downloaded. [Trisha (actress), 36 years, 1.67 m]
1666/1732 Actors downloaded. [Tulip Joshi, 40 years, 1.68 m]
1667/1732 Actors downloaded. [Tun Tun, 80 years, 172 cm tall]
1668/1732 Actors downloaded. [Twinkle Khanna, 46 years, 1.63 m]
1669/1732 Actors downloaded. [Udhayathara, 32 years, None]
1670/1732 Actors downloaded. [Udaya Chandrika, 54 years, 1.69 m]
1671/1732 Actors downloaded. [Udita Goswami, 36 years, 1.7 m]
1672/1732 Actors downloaded. [Ujjwala Raut, 41 years, 1.8 m]
1673/1732 Actors downloaded. [Uma Shankari, 41 years, None]
1674/1732 Actors downloaded. [Uma Padmanabhan, About 55 years, 1.67 m]
1675/1732 Actors downloaded. [Umashree, 62 years, 1.63 m]
1676/1732 Actors downloaded. [Usha Chavan, 64 years, None]
1677/1732 A

###### Check the data downloaded

In [48]:
df = pd.read_csv('./data.csv')
display(df.head(5))

,name,image,age,height,description
0,A. K. Hangal,1.jpg,98 years,1.63 m,Avtar Kishan Hangal (1 February 1914 – 26 Augu...
1,Aadhi Pinisetty,2.jpg,37 years,1.72 m,Aadhi Pinisetty is an Indian film actor who ap...
2,Aadi (actor),3.jpg,30 years,1.68 m,Aadi Saikumar (born Aditya Pudipeddi) is an In...
3,Aamir Khan,4.jpg,55 years,1.63 m,Mohammed Aamir Hussain Khan (pronounced ; born...
4,Ashish Chaudhary,5.jpg,41 years,1.78 m,Ashish Chaudhary (born 21 July 1978) is a Bol...


###### Number of actors without each type of information

In [49]:
ages = df['age'].value_counts()['Unavailable']
heights = df['height'].value_counts()['Unavailable']
des = df['description'].isnull().sum()

print("Total actors: {}".format(df.shape[0]))
print('Number of ages unavailable: {}.\nNumber of heights unavailable: {}.\nNumber of descriptions unavailable: {}'.format(ages, heights, des))

Total actors: 1732
Number of ages unavailable: 102.
Number of heights unavailable: 186.
Number of descriptions unavailable: 2
